# 1. Importation des librairies et chargement du dataset

In [22]:

from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Masking, Embedding, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn import metrics
from sklearn import linear_model
from keras.models import Sequential
import pickle

import requests
import json



nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [24]:
data = pd.read_csv('gdrive/MyDrive/Projet_NLP/dataset.csv',encoding='ISO-8859-1',header=None,names=['target','ids','date','flag','user','text'])

dataset_neg=data['text'][:10000]
dataset_pos=data['text'][800000:810000]

sentiment_neg=data['target'][:10000]
sentiment_pos=data['target'][800000:810000]


# 2. Preprocessing the data

##Mettre en minuscule

In [25]:

def to_lower(data):
  tab_lower=[]
  for string in data:
    tab_lower.append(string.lower())
  
  return tab_lower

dataset_pos_lower=to_lower(dataset_pos)
dataset_neg_lower=to_lower(dataset_neg)

#print(dataset_pos_lower[:5])
#print(dataset_neg_lower[:5])

##Enlever la ponctuation et les chiffres

In [26]:

def to_tab(data):
  sentiment_tab=[]
  for string_line in data:
    sentiment_tab.append(string_line)

  return sentiment_tab

dataset_sentiment_pos = to_tab(sentiment_pos)
dataset_sentiment_neg = to_tab(sentiment_neg)

dataset_sentiment= dataset_sentiment_pos + dataset_sentiment_neg




def dataset_nopunct(data):
  punct = string.punctuation
  tab_punct=[]
  for i in range(len(punct)):
    tab_punct.append(punct[i])


  no_punct_dataset=[]
  element=""
  for elt in data:
    for letter in elt:
      if letter not in tab_punct and not letter.isdigit():
        element += letter

    no_punct_dataset.append(element)
    element =""
  
  return no_punct_dataset

datasetPosNoPunct=dataset_nopunct(dataset_pos_lower)
datasetNegNoPunct=dataset_nopunct(dataset_neg_lower)

#print(datasetPosNoPunct[:5])
#print(datasetNegNoPunct[:5])


##Tokenization des tweets

In [27]:

def Tokenize(data):
  tab_traitement=[]
  for elt in data:
    tab_traitement.append(nltk.word_tokenize(elt))
  
  return tab_traitement

dataset_pos_tokenize = Tokenize(datasetPosNoPunct)
dataset_neg_tokenize = Tokenize(datasetNegNoPunct)

#print(dataset_pos_tokenize[:2])
#print(dataset_neg_tokenize[:2])


##Enlever les stop words

In [28]:

def NoStopWord(data):
  dataset_nostopword=[]
  for tab in data:
    dataset_nostopword.append([w for w in tab if not w in list(nltk.corpus.stopwords.words('english'))])
  return dataset_nostopword

dataset_pos_without_stopwords = NoStopWord(dataset_pos_tokenize)
dataset_neg_without_stopwords = NoStopWord(dataset_neg_tokenize)

#print(dataset_pos_without_stopwords[:2])
#print(dataset_neg_without_stopwords[:2])

##Stemmatization

In [29]:

def Stem(data):
  stemmer = nltk.stem.snowball.EnglishStemmer()
  dataset_pos_stem=[]
  for i in range(len(data)):
    dataset_pos_stem.append([stemmer.stem(w) for w in data[i]])

  return dataset_pos_stem

dataset_pos_stem = Stem(dataset_pos_without_stopwords)
dataset_neg_stem = Stem(dataset_neg_without_stopwords)


#print(dataset_pos_stem[:2])
#print(dataset_neg_stem[:2])

##Lemmatization

In [30]:

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def Lem(data):
  Word_Lemmatizer = WordNetLemmatizer()

  dataset_pos_lema=[]
  for i in range(len(data)):
    dataset_pos_lema.append([Word_Lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in data[i]])
  
  return dataset_pos_lema

dataset_pos_lema = Lem(dataset_pos_stem)
dataset_neg_lema = Lem(dataset_neg_stem)

#print(dataset_pos_lema[:2])
#print(dataset_neg_lema[:2])

##Set des mots uniques

In [31]:
#Mes tweets positifs se retrouvent en premier, dans le dataset c'est l'inverse, pas d'inquiétude
dataset_lema= dataset_pos_lema + dataset_neg_lema

def UniquePos(data):
  tab_unique=set()
  for i in range(len(data)):
    for j in range(len(data[i])):
      tab_unique.add(data[i][j])
  return tab_unique

tab_unique  = UniquePos(dataset_lema)
#print(len(tab_unique),tab_unique)


#Transformation en vecteurs (bag of words)

In [32]:

def VecteurData(data):
  tab_vecteur=[]
  Vecteur=[]


  for j in range(len(data)):
    for word in tab_unique:
      if word in data[j]:
        Vecteur.append(data[j].count(word))
      else:
        Vecteur.append(0)
    tab_vecteur.append(Vecteur)
    Vecteur=[]
  return tab_vecteur

tab_vecteur = VecteurData(dataset_lema)


##Tableau des sentiments et map des targets

In [33]:

#Si le sentiment a pour valeur 4, c'est qu'il est positif, on le change donc en 1 pour notre régression
y = np.array(list(map(lambda x: 1 if x==4 else 0, dataset_sentiment)))
print(len(y),y)

x=tab_vecteur

20000 [1 1 1 ... 0 0 0]


##Sauvegarde du traitement

In [34]:

#pickle_out = open("/content/gdrive/MyDrive/Projet_NLP/Regression_Logistique/x.pickle","wb")
#pickle.dump(x, pickle_out)
#pickle_out.close()

#pickle_out = open("/content/gdrive/MyDrive/Projet_NLP/Regression_Logistique/y.pickle","wb")
#pickle.dump(y, pickle_out)
#pickle_out.close()

#pickle_out = open("/content/gdrive/MyDrive/Projet_NLP/Regression_Logistique/tab_unique.pickle","wb")
#pickle.dump(tab_unique, pickle_out)
#pickle_out.close()


##Fonction de traitement de nos futurs inputs (pipeline)

In [35]:


def traitement(my_string,tab_unique):

  tokenizer = nltk.RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(my_string.lower())
  tokens = [w for w in tokens if not w in list(nltk.corpus.stopwords.words('french'))]

  stemmer = nltk.stem.snowball.EnglishStemmer()
  Word_Lemmatizer = WordNetLemmatizer()
  tokens_stem = [stemmer.stem(w) for w in tokens]

  tokens_lema = [Word_Lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in tokens_stem]


  Vecteur=[]
  
  for word in tab_unique:
    if word in tokens_lema:
      Vecteur.append(tokens_lema.count(word))
    else:
      Vecteur.append(0)
  
 

  return Vecteur



#Chargement des données

In [36]:

#pickle_in = open("/content/gdrive/MyDrive/Projet_NLP/Regression_Logistique/x.pickle","rb")
#X = pickle.load(pickle_in)

#pickle_in = open("/content/gdrive/MyDrive/Projet_NLP/Regression_Logistique/y.pickle","rb")
#Y = pickle.load(pickle_in)

#pickle_in = open("/content/gdrive/MyDrive/Projet_NLP/Regression_Logistique/tab_unique.pickle","rb")
#tab_unique = pickle.load(pickle_in)


#Split de la data et entrainement du modèle de regression logistique

In [37]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)

modele_regLog = linear_model.LogisticRegression(random_state = 0,solver = 'liblinear', multi_class = 'auto')

modele_regLog.fit(x_train,y_train)

precision = modele_regLog.score(x_test,y_test)
print(precision*100)

72.625


#Récupération de data pour tester notre modèle grâce à l'API Yelp

In [38]:
api_key='q1HJoEmRMdypie_E6lYwiih0hNfuSatSTQXH6Bh9fnltm30JX4XO1TSCd6NARRVszrs4lMXb7N3i6LWB_UIBEinyG_ah-0expG6GUnzTdUss2gvNQRcJ7fJhK5g2YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}


url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))


data=json.loads(req.text)
data
 

the status code is 200


{'possible_languages': ['fr', 'en', 'zh', 'pt', 'de', 'it', 'sv', 'ja', 'es'],
 'reviews': [{'id': 'qOiuCTz_Gzgmose1GHaAlg',
   'rating': 5,
   'text': "Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...",
   'time_created': '2020-12-19 07:52:35',
   'url': 'https://www.yelp.com/biz/shake-shack-new-york-2?adjust_creative=KzZVmcfbYqCMY9omoha5NA&hrid=qOiuCTz_Gzgmose1GHaAlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KzZVmcfbYqCMY9omoha5NA',
   'user': {'id': 'FRKnLBnlFiasr1Dc8oOIGQ',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/1wm0yjWSw92j_ZsUFZBGzQ/o.jpg',
    'name': 'Sarah G.',
    'profile_url': 'https://www.yelp.com/user_details?userid=FRKnLBnlFiasr1Dc8oOIGQ'}},
  {'id': '7d56A_ObMPHHyywcfhnrUw',
   'rating': 5,
   'text': 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those c

#Prédiction

In [39]:

#Pour la première phrase test, je prends le premier élément de ma requête,
#mais pour les deux autres j'ai copier coller les reviews complètes car l'API ne renvoie qu'une partie du texte, je voulais tester sur quelque chose de plus long

str1 =data["reviews"][0]["text"]
phrase_pos_test=traitement(str1,tab_unique)

#la str2 ne vient pas de la requête mais du site où str1 est prise, comme ça je peux cibler une review négative
#str2=data["reviews"][1]["text"]
str2="Fool me once shame on you, fool me twice shame on me. We decided to place an order (30 wings) for the Super Bowl this year....bad idea.After 2.5 hours with the phone lines busy we got a call, asking if we still wanted our order.I then remembered we placed an order the year prior and never got our wings from Atomic because we got a message, they ran out of wings, similar to this year.Thanks but no thanks."
phrase_neg_test=traitement(str2,tab_unique)

#str3=data["reviews"][2]["text"]
str3="Buying beware...these are addicting.Still the best doughnuts in my opinion in the city. Ginormous and fluffy like a pillow but somehow still a substantial bite.All doughnuts are scrumptious but greatly enjoyed the Dulce De De Leche in the batch we tried the best.Big win, you can order on Seamless for delivery also on the website they say Caviar, Doordash, Uber Eats and Grub Hub and Sweetist are among other sites for delivery as well."
phrase_pos2_test=traitement(str3,tab_unique)

a = {0:'Négatif', 1:'Positif'}
print(a)


prediction_review = modele_regLog.predict([phrase_pos_test,phrase_neg_test,phrase_pos2_test])
print(prediction_review)

#NORMALEMENT POSITIF
print(str1 + ": ",a[prediction_review[0]])
#NORMALEMENT NEGATIF
print(str2 + ": ",a[prediction_review[1]])
#NORMALEMENT POSITIF
print(str3 + ": ",a[prediction_review[2]])

{0: 'Négatif', 1: 'Positif'}
[1 0 1]
Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...:  Positif
Fool me once shame on you, fool me twice shame on me. We decided to place an order (30 wings) for the Super Bowl this year....bad idea.After 2.5 hours with the phone lines busy we got a call, asking if we still wanted our order.I then remembered we placed an order the year prior and never got our wings from Atomic because we got a message, they ran out of wings, similar to this year.Thanks but no thanks.:  Négatif
Buying beware...these are addicting.Still the best doughnuts in my opinion in the city. Ginormous and fluffy like a pillow but somehow still a substantial bite.All doughnuts are scrumptious but greatly enjoyed the Dulce De De Leche in the batch we tried the best.Big win, you can order on Seamless for delivery also on the website they say Caviar, Doordash, Uber Eats and Grub Hub